## Knn regresija

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
import joblib

# Įkeliame duomenis iš pateikto failo
data = pd.read_csv('data_visi_v2.csv')

# Pasiruošiame duomenis
X = data.drop(columns=['KIEKIS'])
y = data['KIEKIS']

# Padalijame į treniravimo ir testavimo rinkinius
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sukuriame polinominius požymius
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Normalizuojame duomenis
scaler = StandardScaler()
X_train_poly_scaled = scaler.fit_transform(X_train_poly)
X_test_poly_scaled = scaler.transform(X_test_poly)

# Sukuriame K-nn Regressor
knn = KNeighborsRegressor()

# Nustatome hiperparametrų tinklą
param_grid = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan','minkowski']
}

In [2]:
# Naudojame GridSearchCV geriausiems parametrams rasti
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_poly_scaled, y_train)


GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski'],
                         'n_neighbors': [3, 5, 7, 10],
                         'weights': ['uniform', 'distance']},
             scoring='r2')

In [3]:
# Patikrinkime rezultatus ir raskime NaN reikšmes
results = grid_search.cv_results_
mean_test_scores = results['mean_test_score']
params_with_nan = [results['params'][i] for i in range(len(mean_test_scores)) if np.isnan(mean_test_scores[i])]

# Spausdiname kombinacijas, kurios sukelia NaN reikšmes
print("Parametrų kombinacijos, kurios sukelia NaN reikšmes:")
for params in params_with_nan:
    print(params)

Parametrų kombinacijos, kurios sukelia NaN reikšmes:


In [4]:

# Geriausi parametrai
best_params = grid_search.best_params_

# Geriausias modelis
best_model = grid_search.best_estimator_

# Atliekame prognozes su geriausiu modeliu
y_train_pred = best_model.predict(X_train_poly_scaled)
y_test_pred = best_model.predict(X_test_poly_scaled)

# Vertiname modelį
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Kryžminė validacija su pilnu duomenų rinkiniu
X_combined = pd.concat([X_train, X_test])
y_combined = pd.concat([y_train, y_test])
X_combined_poly = poly.fit_transform(X_combined)
X_combined_poly_scaled = scaler.fit_transform(X_combined_poly)

cv_scores_5 = cross_val_score(best_model, X_combined_poly_scaled, y_combined, cv=5, scoring='r2')
cv_scores_10 = cross_val_score(best_model, X_combined_poly_scaled, y_combined, cv=10, scoring='r2')


# Išsaugome modelį
joblib.dump(best_model, 'knn_model.pkl')
# Išsaugome PolynomialFeatures ir StandardScaler
joblib.dump(poly, 'knn_poly.pkl')
joblib.dump(scaler, 'knn_scaler.pkl')


# Rezultatų spausdinimas
print(f'Geriausi parametrai: {best_params}')
print(f'Treniravimo MSE: {train_mse}')
print(f'Testavimo MSE: {test_mse}')
print(f'Treniravimo R2: {train_r2}')
print(f'Testavimo R2: {test_r2}')

print(f'Kryžminės validacijos (cv=5) MSE reikšmės: {cv_scores_5}')
print(f'Kryžminės validacijos (cv=5) vidutinis MSE: {cv_scores_5.mean()}')

print(f'Kryžminės validacijos (cv=10) MSE reikšmės: {cv_scores_10}')
print(f'Kryžminės validacijos (cv=10) vidutinis MSE: {cv_scores_10.mean()}')


Geriausi parametrai: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}
Treniravimo MSE: 1179824.4180357144
Testavimo MSE: 1655609.1789473684
Treniravimo R2: 0.9574232460729571
Testavimo R2: 0.9381141459500063
Kryžminės validacijos (cv=5) MSE reikšmės: [0.96267899 0.91710336 0.96068257 0.95384504 0.93632822]
Kryžminės validacijos (cv=5) vidutinis MSE: 0.9461276374364201
Kryžminės validacijos (cv=10) MSE reikšmės: [0.9712934  0.95091006 0.80813735 0.96734087 0.94249511 0.97162942
 0.96015372 0.95592331 0.95008886 0.93269786]
Kryžminės validacijos (cv=10) vidutinis MSE: 0.9410669958481966


In [ ]:
# Rezultatai rodo, kad modelis veikia gana gerai, 
# tačiau yra tam tikrų skirtumų tarp treniravimo ir testavimo duomenų, 
# taip pat kryžminės validacijos rezultatų.

# Geriausi parametrai: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}
# Naudojama euklido atstumo metrika.
# Kaimynų skaičius  k yra 5.
# Svorio funkcija yra "uniform", tai reiškia, kad visi kaimynai turi vienodą svorį prognozėje.

# Treniruotės ir Testavimo rezultatai
# Treniravimo MSE: 1179824.4180357144
# Testavimo MSE: 1655609.1789473684
# Treniravimo R2: 0.9574232460729571
# Testavimo R2: 0.9381141459500063

# Treniravimo MSE yra 1179824.418, o Testavimo MSE yra 1655609.179. 
# Nors testavimo klaida yra šiek tiek didesnė, jos vertė yra artima treniravimo klaidai, 
# kas rodo, kad modelis nėra per daug pritaikytas (overfit).

# Treniravimo 𝑅2 yra 0.957, o Testavimo 𝑅2 yra 0.938.
# Abu šie skaičiai yra aukšti ir rodo, kad modelis gerai paaiškina duomenų dispersiją 
# tiek treniravimo, tiek testavimo rinkiniuose.

# Kryžminė validacija
# Kryžminės validacijos (cv=5) MSE reikšmės: [0.96267899 0.91710336 0.96068257 0.95384504 0.93632822]
# Kryžminės validacijos (cv=5) vidutinis MSE: 0.9461276374364201
# Kryžminės validacijos su 5 dalimis (cv=5) MSE reikšmės svyruoja tarp 0.917 ir 0.962.
# Vidutinis MSE yra 0.946, tai rodo stabilų modelio veikimą kryžminėje validacijoje su 5 dalimis.

# Kryžminės validacijos (cv=10) MSE reikšmės: [0.9712934  0.95091006 0.80813735 0.96734087 0.94249511 0.97162942
# 0.96015372 0.95592331 0.95008886 0.93269786]
# Kryžminės validacijos (cv=10) vidutinis MSE: 0.9410669958481966
# Kryžminės validacijos su 10 dalimis (cv=10) MSE reikšmės svyruoja tarp 0.808 ir 0.971.
# Vidutinis MSE yra 0.941, tai rodo stabilų modelio veikimą kryžminėje validacijoje su 10 dalimis.

# Išvados
# Modelio veikimas yra geras, ką rodo aukštas 𝑅2 tiek treniravimo, tiek testavimo rinkiniuose.
# MSE yra didesnis testavimo rinkinyje nei treniravimo rinkinyje, bet skirtumas nėra labai didelis, 
# kas rodo, kad modelis nėra per daug pritaikytas (overfit).
# Kryžminės validacijos rezultatai rodo stabilų modelio veikimą įvairiuose duomenų rinkiniuose.
# Taigi, modelis gali būti laikomas geru pasirinkimu prognozėms, 
# tačiau reikia atkreipti dėmesį į didesnę MSE vertę testavimo duomenų rinkinyje, 
# kuri gali rodyti galimus duomenų svyravimus ar netikslumus, kuriuos verta toliau analizuoti.
